# JupyterNotebook de la App para Electrobuzz

## Intro

En este cuaderno se pretende ir añadiendo todos los avances relativos al proyecto del desarrollo de la App para la descarga de música mediante Electrobuzz-Cosmobox. Cabe mencionar que este proyecto no está pensado para realizar un web-scrapping ilícito. Lo que se pretende mediante este programa es agilizar el proceso de descarga de música de Electrobuzz, pero teniendo en cuenta que es necesario contar con una cuenta de Cosmobox para ello. Tambien se pretende aprender a programar un scraper en python y hacer una pequeña interfaz de usuario que pueda acabar en convertirse en un ejecutable, QUIERO APRENDER A HACER SOFTWARE, PERO SOFTWARE CACHARRO. 

## Fundamentos

En principio esta app se basará en realizar peticiones mediante el paquete requests. Se ha planteado realizar el trabajo inicial en 3 rondas de peticiones. 

En primera instancia todos las urls de todos los albumes disponibles en Electrobuzz. 
Una vez se consiga tener un listado de todos los links se pretende realizar una seguna ronda de requests que se encarge de completar la información de cada album, asi como Artista, Remixers, tamaño de la descarga y SOBRE TODO comprobar que exista el link de descarga de Cosmobox (existen albumes que no estan disponibles para descargar en Cosmobox y por lo tanto no son accesibles para nosotros). 
En la tercera ronda de requests se pretende comprobar que el link de Cosmobox no está caído (Hay  bastantes links que no funcionan).

Con toda esta informacion se pasará a crear una interfaz empleando el paquete tkinter, la cual debe contar con un motor de busqueda dentro de toda la información de electrobuzz que nos permita rápidamente seleccionar los albumes que se quieren descargar. Sería interensante que dentro de este motor de busqueda se pudiera filtrar por nombre, sello, fecha, etc, para agilizar la labor de buscar la mñusica que nos apetezca. 

Dentro de la interfaz de tkinter habrá un botón para actualizar la información contenida en el software, como un método para tener al día todos los realeases de la página... Aunque tambien se podría realizar una actualización de manera automática. 

Debe ser sencillo selecionar la música que queremos descargar, igual añadimos un click marker y un botón de descarga al final de la página. Ya veré. Este paso no se si se realizará con requests (espero que si) o con Selenium... ambos tienen sus ventajas y desventajas. 


A continuación voy a añadir las librerias que voy empleando. Destacando requests y BeautifulSoup. 






In [24]:
from bs4 import BeautifulSoup
import requests

import shutil
import numpy as np
import os
import random

## Problemas detectados

El primero y yo creo que más importante es que los cabrones de Electrobuzz no dejan realizar scraping... Esto se puede confirmar viendo el archivo robot.txt de la página web. 

![Robot.txt de la página Electrobuzz](img_jupyter/robot.png)

El asterísco en User-Agent siginifica que no se permiten spyders, crawlers y compañia. Por lo tanto intentarán bloquear cualquier comportamiento que sea similar al de un boot, por ejemplo, muchas requests en poco tiempo, o muchas requests iguales. Una vez han detectado a un boot lo siguiente que hacen es bloquear la IP que está realizando las peticiones. La IP la marca tu router, pa entendernos, por lo tanto hay que usar VPN o proxies. Investigando un poquito llegue a la conclusión de que iba a ser más fácil incluir el método de cambiar de proxies, sobre todo porque el paquete requests ya incluye métodos para fijar una proxie. 

## Buscar y cambiar proxy

Para esto es necesario saber que las proxies no son fijas, es decir, las proxies son puntos de conexion que no siempre se encuentran disponibles por ello es necesario ir actualizando la lista de proxies a las que vamos a conectarnos. Para ello yo voy a usar una página web que ofrece un servicio de proxies gratuitas. 

Creo una función que devuelve las proxies encontradas en la página web. 

In [25]:
def get_proxies():   
   
  proxies_req = requests.get('https://www.sslproxies.org/')


  soup = BeautifulSoup(proxies_req.content, 'html.parser')
  proxies_table = soup.find(id='proxylisttable')
  
  proxies=[]

  # Save proxies in the array
  for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

  return proxies



get_proxies() [:3]

[{'ip': u'82.85.185.185', 'port': u'49788'},
 {'ip': u'182.253.101.59', 'port': u'51124'},
 {'ip': u'79.142.63.186', 'port': u'63141'}]

Es necesario entender como se deben incluir las proxies a las requests. Sin más, te lo demuestro con la función que he creado para poner en formato las proxies par que las entienda el paquete request. 

A esta función hay que meter como argumentos la lista de proxies que devuelte get_proxies() y el número dentro de la lista. 

In [26]:
def set_proxi_for_req(proxies,n):
    proxi_http='http://' + str(proxies[n]['ip'])+ ':' +proxies[n]['port']
    proxi_https='https://' + str(proxies[n]['ip'])+ ':' +proxies[n]['port']
    proxi1 = {'http': str(proxi_http), "https": str(proxi_https)}
    return proxi1




proxies=get_proxies()

set_proxi_for_req(proxies, 5)

{'http': 'http://118.173.233.134:60622',
 'https': 'https://118.173.233.134:60622'}

Ese es el formato que requiere el paquete requests para fijar las proxies y se introduce la siguiente manera. 

In [27]:
%%capture capt
requests.get('url', proxies= 'proxi en formato visto')


MissingSchema: Invalid URL 'url': No schema supplied. Perhaps you meant http://url?

Nosotras lo vamos a utilizar asi

In [28]:
requests.get('url', proxies= set_proxi_for_req(proxies,16))

MissingSchema: Invalid URL 'url': No schema supplied. Perhaps you meant http://url?